# WEB SCRAPING PRACTICE - DATA NORMALIZATION

## Libraries

In [105]:
import pandas as pd
from bs4 import BeautifulSoup # Librería para crear objetos tipo soup
import urllib3 # Librería para descargar el contenido de una URL definida
import numpy as np # Librería para manipular arreglos
from sqlalchemy import create_engine # Librería para crear las conexiones con la base de datos Postgre
from datetime import datetime
from datetime import timedelta, time
import pickle
import requests
import sys
import re
import random # Librería para generar números randómicos

## Parameters

In [98]:
# Variable para crear la conexión sobre el ambiente "sandbox"
env = "Sandbox"
frecuency = "daily" 

## Connections for PostgreSQL Database

In [10]:
with open('../settings/hotDatamart{}.pkl'.format(env),'rb') as f:  
    psqlUser,psqlPass,psqlHost,psqlPort,psqlDatabase = pickle.load(f)
    
psqlConnection = create_engine('postgresql://{}:{}@{}:{}/{}'.format(psqlUser,psqlPass,psqlHost,psqlPort,psqlDatabase),echo=False)

## Functions

In [11]:
 
##*********************** FUNCIONES PARA NORMALIZACIÓN DE DATOS DE AMAZON *********************

# Función para efectuar la conversión de símbolo a moneda local de cada país
def getCurrency(x): 
    if ("€" in x):
        return ("Euro")
    elif ("$" in x):
        return ("Dólar")
    else:
        return ("Libra Esterlina")

# Extracción del precio del campo "price"
def getPrice(x): 
    # Aplicamos dos condiciones ya que el precio viene con coma o punto
    if ("," in x):
        temp = re.findall(r'\d+\,\d+', x)
        return temp
    else:
        temp = re.findall(r'\d+\.\d+', x) 
        return temp

# Reemplazo del punto por la coma donde corresponda  
def replaceComma(x): 
    if ("," in x):
        x = x.replace(',','.')
        return float(x)
    else:
        return float(x)


# Extracción del dato desde la lista
def numConversion(x): 
    temp = x[0]
    return temp

##*********************** FUNCIONES PARA NORMALIZACIÓN DE DATOS DE CORONAVIRUS *********************

# Función especial para automatizar la conversión de caracteres especiales y limpieza de datos de los datos sobre coronavirus 
def revisa_columna(x): 
    
    if (x!="" and x!=" " and x!="  "): 
        x = x.strip()
        x = x.replace("+", "")
        x = x.replace(",", "")
        x = x.replace(".", "")
        x = int(x)       
                  
    else:
        x = 0
               
    return x

# AMAZON DATA NORMALIZATION

In [ ]:
''' 
El dataframe obtenido de la extracción de datos de la web de Amazon "dailyData" pasa por el siguiente proceso de normalización de datos.

El conjunto de datos contiene los siguientes campos normalizados:

No hay N/A detectados

'asin',              tipo CHR          "Amazon Standar Identification Number - Código único de identificación del producto"
'description',       tipo CHR          "Descripción del producto"
'dateTime'           tipo timestamp    "Fecha y hora en la que se realiza la captura de datos"
'date'               tipo datetime     "Fecha en la que se realiza la captura de datos"
'country'            tipo CHR          "País al que pertenece el producto "
'currency'           tipo CHR          "Moneda local en la que se expresa el precio del producto"
'price'              tipo FLOAT        "Precio del producto"

Finalmente, se crean conexiones con una base PostgreSQL donde se los almacena durante el periodo de extracción definido.

'''

## Amazon Data normalization

In [14]:
## Normalización de datos de AMAZON
dailyData = dailyData.reset_index(drop=True)

# Aplicamos la función getCurrency() para obtener la moneda local de cada país
dailyData['currency'] = dailyData['price'].apply(getCurrency)

# Obtenemos el precio que se encuentra dentro del string "price" y queda en una lista
dailyData['price2'] = dailyData['price'].apply(getPrice)

# Extraemos el precio de la lista y lo dejamos como string
dailyData['price2'] = dailyData['price2'].apply(numConversion)

# Reeplazamos el punto por la coma donde fuere necesario
dailyData['price2'] = dailyData['price2'].apply(replaceComma)

# Convertimos a formato flotante
dailyData['price2'] = dailyData['price2'].astype(float)

# Eliminamos el precio inicial y renombramos la columna procesada
dailyData = dailyData.drop(['price'], axis=1, errors='ignore')
dailyData = dailyData.reset_index(drop=True)
dailyData = dailyData.rename(columns={'price2': 'price'})

## Save Amazon Data in postgre database

In [15]:
dailyData.to_sql('{}_am_pr_detailed_final'.format(frecuency), con=psqlConnection, schema='platform' ,if_exists='append', index=False)

## Save Amazon Data in .csv for backup

In [17]:
# Si el fichero ya está en el repositorio hacemos la lectura del mismo
datos_disco_amazon = pd.read_csv("/home/jovyan/mylab/UOC/Respaldo/datos_amazon.csv", sep=";")
# Cremos un nuevo DF con la contatenación de lo que existía en el repositorio y lo generado en esta iteración.
backup_stack = pd.concat([datos_disco_amazon, dailyData], axis=0)
# Exportamos como un fichero CSV ACUMULADO los datos del DF con los nuevos ficheros añadidos
backup_stack.to_csv('/home/jovyan/mylab/UOC/Respaldo/datos_amazon.csv', sep=";", index=False)

# CORONAVIRUS DATA NORMALIZATION

In [1]:
''' 
El dataframe obtenido de la extracción de datos de la web de Worldometers "datos_pandemia_df" pasa por el siguiente proceso de normalización de datos.

El conjunto de datos contiene los siguientes campos normalizados:

Datos de INT en unidades.

No hay N/A detectados. Los valores no detectados son CERO.

'dateTime'           tipo timestamp    "Fecha y hora en la que se realiza la captura de datos"
'date'               tipo datetime     "Fecha en la que se realiza la captura de datos"
'country',           tipo CHR          "País al que hacen referencia los datos, salvo un crucero"
'total_cases'        tipo INT          "Número total de casos que se han contabilizado en el país"
'new_cases'          tipo INT          "Número total de casos incrementados desde la info anterior en el país"
'total_deaths'       tipo INT          "Número total de fallidos en el país "
'new_deaths'         tipo INT          "Número total de fallacidos incrementados desde la info anterior en el país"
'total_recovered'    tipo INT          "Número total de recuperados en el país "
'active_cases'       tipo INT          "Número total de casos en tratamiento en el país "
'critical_cases'     tipo INT          "Número total en tratamiento en UCI en el país "
'cases_per_million'  tipo INT          "Ratio de casos por millón de habitantes en el país "
'deaths_per_million' tipo INT          "Ratio de muertes por millón de habitantes en el país "

Finalmente, se crean conexiones con una base PostgreSQL donde se los almacena durante el periodo de extracción definido.

'''

' \nEl dataframe resultante de la extracción de datos de la web de Amazon "dailyData" pasa por el siguiente proceso de normalización de datos.\n\nFinalmente, se crean conexiones con una base PostgreSQL donde se los almacena durante el periodo de extracción definido.\n\n'

## Coronavirus Data normalization

In [23]:
# Bloque de conversión de tipos y normalización de campos de DF que contiene los datos.

# Creamos una funcion ad.hoc para cambiar el formato de las columnas a string
datos_pandemia_df = datos_pandemia_df.replace(to_replace=r'', value=0, regex=True)
datos_pandemia_df[['country','total_cases','new_cases','total_deaths','new_deaths','total_recovered','active_cases','critical_cases','cases_per_million','deaths_per_million']] = datos_pandemia_df[['country','total_cases','new_cases','total_deaths','new_deaths','total_recovered','active_cases','critical_cases','cases_per_million','deaths_per_million']].astype(str) 

  
# Definimos las columnas que tienen que tener revisiones o cambios de formato.
columnas2 = [ 'total_cases','new_cases','total_deaths','new_deaths','total_recovered','active_cases','critical_cases','cases_per_million','deaths_per_million']

# Aplicamos la función anterior de limpieza a todo el DF de capturas.
for n in columnas2:
    datos_pandemia_df[n] = datos_pandemia_df[n].apply(revisa_columna)
    
datos_pandemia_df["date"] = datos_pandemia_df["dateTime"].apply(lambda x: x.date())

## Save Coronavirus Data in postgre database

In [24]:
datos_pandemia_df.to_sql('{}_corona_detailed_final'.format(frecuency), con=psqlConnection, schema='platform' ,if_exists='append', index=False)

## Save Amazon Data in .csv for backup

In [26]:
# Si el fichero ya está en el repositorio hacemos la lectura del mismo
datos_disco_df = pd.read_csv("/home/jovyan/mylab/UOC/Respaldo/datos_pandemia.csv", sep=";")
# Cremos un nuevo DF con la contatenación de lo que existía en disco y lo generado en esta ejecución.
vertical_stack = pd.concat([datos_disco_df, datos_pandemia_df], axis=0)
# Exportamos como un fichero CSV ACUMULADO los datos del DF con los nuevos ficheros añadidos
vertical_stack.to_csv('/home/jovyan/mylab/UOC/Respaldo/datos_pandemia.csv', sep=";", index=False)

# BACKGROUND RESEARCH

## Technology used by the websites

In [27]:
import builtwith
builtwith.parse('https://www.worldometers.info/coronavirus/')
builtwith.parse('https://www.amazon.com/')

ModuleNotFoundError: No module named 'builtwith'

## Owner of the website

In [ ]:
import whois
print(whois.whois('https://www.amazon.com/'))
print(whois.whois('https://www.worldometers.info/'))

## Parsing robots.txt

In [ ]:
import robotparser

# Módulo robotparser para interpretar el fichero robots.txt de Amazon
rp = robotparser.RobotFileParser()
rp.set_url('https://www.amazon.com/robots.txt')
rp.read()

In [ ]:
# Validación de "User Agent"
url = 'https://www.amazon.com'
user_agent = 'BadCrawler' 

## Método can_fetch() para conocer si un "user agent" en particular tiene o no el acceso permitido a una página web
rp.can_fetch(user_agent, url) 
user_agent = 'GoodCrawler' 
rp.can_fetch(user_agent, url) 
user_agent = 'Googlebot' 
rp.can_fetch(user_agent, url) 